In [ ]:
# import torch
# import numpy as np
# import cv2
# from time import time
# from ultralytics import YOLO
# import supervision as sv

# class ObjectDetection:

#     def __init__(self, capture_index):
       
#         self.capture_index = capture_index
        
#         self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
#         print("Using Device: ", self.device)
        
#         self.model = self.load_model()
        
#         self.CLASS_NAMES_DICT = self.model.model.names
    
#         self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=1)
    

#     def load_model(self):
       
#         model = YOLO("runs\\detect\\train4\\weights\\best.pt")  # enter path for a custom trained YOLOv8n model
#         model.fuse()
    
#         return model


#     def predict(self, frame):
#         results = self.model(frame)
#         if isinstance(results, list):
#             results = results[0]
#         print(type(results))
#         return results
    

#     def plot_bboxes(self, results, frame):

#         # Setup detections for visualization
#         detections = sv.Detections(
#                 xyxy=results[0]['boxes'].xyxy.cpu().numpy(),
#                 confidence=results[0]['scores'].cpu().numpy(),
#                 class_id=results[0]['labels'].cpu().numpy().astype(int),
#                 )


#         # Format custom labels
#         self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.7f}"
#         for confidence, class_id in zip(detections.confidence, detections.class_id)]

#         # Annotate and display frame
#         frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)

#         return frame


#     def __call__(self):
    
#         cap = cv2.VideoCapture(self.capture_index)
    
#         assert cap.isOpened()
#         cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
#         cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

#         detect_interval = 3 # interval in seconds between object detections
#         bbox_duration = 3 # duration in seconds to keep the bounding box

#         last_detection_time = 0
#         current_time = 0
#         bbox_start_time = None

#         while True:
            
#             ret, frame = cap.read()
            
#             assert ret

#             current_time = time()

#             if current_time - last_detection_time >= detect_interval:
#                 # perform object detection
#                 results = self.predict(frame)

#                 # check if any objects are detected
#                 if len(results) > 0 and len(results[0]['xyxy']) > 0 and 'label' in results[0]:

#                     last_detection_time = current_time

#                     # set the start time for the bounding box display
#                     bbox_start_time = current_time

#                 # display the bounding box if the duration is not exceeded
#                 if bbox_start_time is not None and current_time - bbox_start_time <= bbox_duration:
#                     frame = self.plot_bboxes(results, frame)
            
#             # display FPS
#             end_time = time()
#             fps = 1/np.round(end_time - current_time, 2)
#             cv2.putText(frame, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
           
#             cv2.imshow('YOLOv8 Detection', frame)

#             if cv2.waitKey(5) & 0xFF == 27:
            
#                 break

#         cap.release()
#         cv2.destroyAllWindows()


# detector = ObjectDetection(capture_index=0)
# detector()

In [ ]:
#폴더 이미지 추론 및 저장, 컨피던스 csv로 저장됨
import torch
import numpy as np
import cv2
import os
import csv
from time import time
from ultralytics import YOLO
import supervision as sv

class ObjectDetection:

    def __init__(self, image_directory, keyframe_interval):
        self.image_directory = image_directory
        self.keyframe_interval = keyframe_interval
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.5)
        self.keyframe_detected = False
        self.confidence_values = []
    

    def load_model(self):
        model = YOLO("best.pt")  # enter path for a custom trained YOLOv8n model
        model.fuse()
        return model


    def predict(self, frame):
        results = self.model(frame)
        return results
    

    def plot_bboxes(self, results, frame):
        # Setup detections for visualization
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
        )
    
         # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        
        return frame
    

    def __call__(self):
        # Specify the directory to save the images
        save_directory = 'fishresult'

        # Check if the directory exists, if not, create it
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        frame_number = 0

        for file_name in os.listdir(self.image_directory):
            if file_name.endswith('.jpg'):
                start_time = time()

                image_path = os.path.join(self.image_directory, file_name)
                frame = cv2.imread(image_path)
                assert frame is not None

                if self.keyframe_detected:
                    self.keyframe_detected = False
                    results = self.predict(frame)
                    frame = self.plot_bboxes(results, frame)

                end_time = time()
                elapsed_time = end_time - start_time
                if elapsed_time > 0:
                    fps = 1 / elapsed_time
                else:
                    fps = float('inf')

                cv2.putText(frame, f'FPS: {int(fps)}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                cv2.imshow('YOLOv8 Detection', frame)

                # Save frames as images
                save_path = os.path.join(save_directory, f'frame_{frame_number}.jpg')
                cv2.imwrite(save_path, frame)
                print(f'Saved frame {frame_number} as {save_path}')

                # Append confidence value to the list
                results = self.predict(frame)
                confidence = results[0].boxes.conf.cpu().numpy()
                self.confidence_values.append(confidence)

                frame_number += 1

                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
                    
        # Save confidence values as a CSV file
        save_path = os.path.join(save_directory, 'confidence.csv')
        with open(save_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['Frame Number', 'Confidence'])
            for i, confidence in enumerate(self.confidence_values):
                writer.writerow([i, confidence])

        cv2.destroyAllWindows()

detector = ObjectDetection(image_directory=r'C:\Users\USER\AI\ultralytics\datasetfishlens\fishlenswithoutcarli', keyframe_interval=1)
detector()


In [ ]:
import numpy as np
import cv2

def calibrate_camera(chessboard_size):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    pattern_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    pattern_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    
    obj_points = []
    img_points = []
    
    cap = cv2.VideoCapture(0)  # 웹캠 영상을 받아오기 위해 0을 인자로 전달합니다.

    while True:
        ret, frame = cap.read()

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret:
            obj_points.append(pattern_points)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            img_points.append(corners)

            frame = cv2.drawChessboardCorners(frame, chessboard_size, corners, ret)

        cv2.imshow('Chessboard', framqe)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    return camera_matrix, dist_coeffs

# 체스보드 크기 (내부 코너 개수)
chessboard_size = (9, 6)

# 카메라 캘리브레이션 수행
camera_matrix, dist_coeffs = calibrate_camera(chessboard_size)

# 결과 출력
print("Camera Matrix:")
print(camera_matrix)
print("\nDistortion Coefficients:")
print(dist_coeffs)


In [ ]:
#캘리브레이션 계산
import numpy as np
import cv2

def calibrate_camera(chessboard_size, image_count):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    pattern_points = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    pattern_points[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    
    obj_points = []
    img_points = []
    image_index = 0

    image_list = ['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg']

    image_index = 0
    image_count = len(image_list)
    
    while image_index < image_count:
        # Load chessboard image from file
        frame = cv2.imread(image_list[image_index])
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, chessboard_size, None)

        if ret:
            obj_points.append(pattern_points)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            img_points.append(corners)

            frame = cv2.drawChessboardCorners(frame, chessboard_size, corners, ret)

            cv2.imshow('Chessboard', frame)
            key = cv2.waitKey(500)  # Delay between capturing each image
            
            if key == ord('q'):  # Press 'q' to exit the loop
                break

        image_index += 1

    cv2.destroyAllWindows()

    ret, camera_matrix, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

    return camera_matrix, dist_coeffs

# 체스보드 크기 (내부 코너 개수)
chessboard_size = (9, 6)
# 캘리브레이션에 사용할 이미지 수
image_count = 10

# 카메라 캘리브레이션 수행
camera_matrix, dist_coeffs = calibrate_camera(chessboard_size, image_count)

# 결과 출력
print("Camera Matrix:")
print(camera_matrix)
print("\nDistortion Coefficients:")
print(dist_coeffs)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os

# 이미지 파일 경로
image_paths = ['1.jpg', '2.jpg', '3.png']
output_folder = 'result_images'  # 결과 이미지 저장 폴더

# 결과 이미지 저장 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 히스토그램 데이터 저장을 위한 CSV 파일 경로
csv_file = os.path.join(output_folder, 'histogram_data.csv')

# 히스토그램 데이터를 저장할 리스트
histogram_data = []

for path in image_paths:
    # 이미지 로드
    image = cv2.imread(path)

    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 히스토그램 계산
    hist = cv2.calcHist([gray], [0], None, [256], [0, 256])

    # 히스토그램 데이터를 리스트에 추가
    histogram_data.append(hist.flatten())

    # 히스토그램 시각화
    plt.figure()
    plt.title('Histogram')
    plt.xlabel('Bins')
    plt.ylabel('# of Pixels')
    plt.plot(hist)
    plt.xlim([0, 256])
    plt.savefig(os.path.join(output_folder, f'{path}_histogram.png'))
    plt.close()

    # 히트맵 생성
    heatmap = cv2.applyColorMap(gray, cv2.COLORMAP_JET)

    # 히트맵 이미지와 원본 이미지를 가로로 연결
    merged = np.hstack((image, heatmap))

    # 이미지 저장
    cv2.imwrite(os.path.join(output_folder, f'{path}_heatmap.png'), merged)

    # 이미지 출력
    cv2.imshow('Merged Image', merged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 히스토그램 데이터를 CSV 파일로 저장
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(histogram_data)


In [1]:
#칼리브레이션 윈도우사이즈 640*480 으로 수정한코드

import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
import supervision as sv
import ctypes

class ObjectDetection:
    def __init__(self, capture_index, keyframe_interval, camera_matrix, dist_coeffs):
        self.capture_index = capture_index
        self.keyframe_interval = keyframe_interval
        self.K = camera_matrix
        self.D = dist_coeffs

        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device:", self.device)

        self.model = self.load_model()

        self.CLASS_NAMES_DICT = self.model.model.names

        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.2)

        self.keyframe_detected = False

         # Set the desired window size
        self.window_width = 640
        self.window_height = 480
        self.set_window_size()

        

    def set_window_size(self):
        # Get the current screen width and height
        user32 = ctypes.windll.user32
        user32.SetProcessDPIAware()
        screen_width = user32.GetSystemMetrics(0)
        screen_height = user32.GetSystemMetrics(1)

        # Calculate the window position and size
        pos_x = (screen_width - self.window_width) // 2
        pos_y = (screen_height - self.window_height) // 2

        # Set the window size and position
        cv2.namedWindow('YOLOv8 Detection', cv2.WINDOW_NORMAL)
        cv2.resizeWindow('YOLOv8 Detection', self.window_width, self.window_height)
        cv2.moveWindow('YOLOv8 Detection', pos_x, pos_y)

    def load_model(self):
        model = YOLO("best.pt")  # enter path for a custom trained YOLOv8n model
        model.fuse()
        return model

    def predict(self, frame):
        results = self.model(frame)
        return results

    def plot_bboxes(self, results, frame):
        # Setup detections for visualization
        detections = sv.Detections(
            xyxy=results[0].boxes.xyxy.cpu().numpy(),
            confidence=results[0].boxes.conf.cpu().numpy(),
            class_id=results[0].boxes.cls.cpu().numpy().astype(int),
        )

        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.7f}"
                       for _, confidence, class_id, tracker_id
                       in detections]

        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)

        return frame

    def undistort(self, frame):
        h, w = frame.shape[:2]
        newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(self.K, self.D, (w, h), 1, (w, h))
        frame = cv2.undistort(frame, self.K, self.D, None, newcameramatrix)
        x, y, w, h = roi
        frame = frame[y:y + h, x:x + w]
        return frame

    def __call__(self):
        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.window_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.window_height)

        image_index = 0  # Counter for saving frames

        while True:
            start_time = time()

            ret, frame = cap.read()
            assert ret

            frame = self.undistort(frame)  # 어안렌즈 보정

            if self.keyframe_detected or cap.get(cv2.CAP_PROP_POS_FRAMES) % self.keyframe_interval == 0:
                self.keyframe_detected = True
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)
            else:
                self.keyframe_detected = False

            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)

            # FPS 표시
            # cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

            cv2.imshow('YOLOv8 Detection', frame)

            # Save each frame as an image
            cv2.imwrite(f'frame_{image_index}.png', frame)
            image_index += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()


focal_length = 1000  # 카메라메트릭스로 유추
principal_point_x = 960  # 카메라메트릭스로 유추
principal_point_y = 540  # 카메라 메트릭스로 유추
k1, k2, p1, p2, k3, k4, k5, k6 = 0, 0, 0, 0, 0, 0, 0, 0  # 예시로 임의로 설정한 값

camera_matrix = np.array([[434.65221747,0.,304.1506615 ],
 [  0.,445.46687396, 291.03864346],
 [  0., 0.,1. ]]
)
dist_coeffs = np.array([[-0.72503986 , 0.62356206 , 0.00102756 , 0.01607157, -0.28152773]])

detector = ObjectDetection(capture_index=0, keyframe_interval=5, camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)
detector()


Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Using Device: cpu



0: 288x640 (no detections), 78.0ms
Speed: 1.1ms preprocess, 78.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 75.2ms
Speed: 0.0ms preprocess, 75.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 60.6ms
Speed: 1.0ms preprocess, 60.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 95.5ms
Speed: 0.0ms preprocess, 95.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 95.9ms
Speed: 0.0ms preprocess, 95.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.4ms
Speed: 1.0ms preprocess, 63.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 60.0ms
Speed: 0.0ms preprocess, 60.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.7ms
Speed: 1.0ms preprocess, 63.7ms 


0: 288x640 1 photoshoot, 74.9ms
Speed: 1.0ms preprocess, 74.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 59.3ms
Speed: 1.0ms preprocess, 59.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 61.3ms
Speed: 0.0ms preprocess, 61.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 62.5ms
Speed: 0.6ms preprocess, 62.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 52.3ms
Speed: 0.0ms preprocess, 52.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 67.4ms
Speed: 0.0ms preprocess, 67.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 55.2ms
Speed: 1.0ms preprocess, 55.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 64.5ms
Speed: 0.0ms preprocess, 64.5ms inference, 0.0ms postproc

0: 288x640 1 photoshoot, 60.6ms
Speed: 0.0ms preprocess, 60.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 58.3ms
Speed: 0.0ms preprocess, 58.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 65.9ms
Speed: 1.0ms preprocess, 65.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 53.2ms
Speed: 1.0ms preprocess, 53.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 68.0ms
Speed: 1.0ms preprocess, 68.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 71.5ms
Speed: 0.0ms preprocess, 71.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 56.3ms
Speed: 1.0ms preprocess, 56.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 64.9ms
Speed: 0.0ms preprocess, 64.9ms inference, 0.0ms postproce

...


0: 288x640 (no detections), 61.5ms
Speed: 0.0ms preprocess, 61.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 57.6ms
Speed: 1.0ms preprocess, 57.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 61.8ms
Speed: 0.0ms preprocess, 61.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.2ms
Speed: 0.0ms preprocess, 63.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 75.4ms
Speed: 0.0ms preprocess, 75.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 74.7ms
Speed: 1.0ms preprocess, 74.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.5ms
Speed: 0.0ms preprocess, 63.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 53.1ms
Speed: 1.0ms preprocess, 53.1ms i


0: 288x640 (no detections), 59.6ms
Speed: 0.0ms preprocess, 59.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 51.2ms
Speed: 10.5ms preprocess, 51.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 64.0ms
Speed: 0.0ms preprocess, 64.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.4ms
Speed: 1.5ms preprocess, 63.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 59.8ms
Speed: 0.0ms preprocess, 59.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 67.6ms
Speed: 0.0ms preprocess, 67.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 66.0ms
Speed: 1.0ms preprocess, 66.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 76.9ms
Speed: 0.0ms preprocess, 76.9ms 


0: 288x640 (no detections), 75.9ms
Speed: 0.0ms preprocess, 75.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 61.1ms
Speed: 1.0ms preprocess, 61.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 65.4ms
Speed: 1.0ms preprocess, 65.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 1 photoshoot, 75.1ms
Speed: 0.0ms preprocess, 75.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 69.1ms
Speed: 0.9ms preprocess, 69.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 77.1ms
Speed: 0.0ms preprocess, 77.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 62.2ms
Speed: 0.0ms preprocess, 62.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 69.3ms
Speed: 0.9ms preprocess, 69.3ms infe


0: 288x640 (no detections), 60.9ms
Speed: 0.0ms preprocess, 60.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 67.8ms
Speed: 0.0ms preprocess, 67.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.6ms
Speed: 0.0ms preprocess, 63.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.2ms
Speed: 0.9ms preprocess, 63.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 62.1ms
Speed: 0.0ms preprocess, 62.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 77.6ms
Speed: 0.0ms preprocess, 77.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 58.5ms
Speed: 0.0ms preprocess, 58.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 79.8ms
Speed: 0.0ms preprocess, 79.8ms i


0: 288x640 (no detections), 62.6ms
Speed: 1.1ms preprocess, 62.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 60.3ms
Speed: 0.0ms preprocess, 60.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 59.4ms
Speed: 0.0ms preprocess, 59.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 63.0ms
Speed: 0.0ms preprocess, 63.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 68.9ms
Speed: 0.0ms preprocess, 68.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 64.2ms
Speed: 1.5ms preprocess, 64.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 54.2ms
Speed: 0.0ms preprocess, 54.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 288x640 (no detections), 62.8ms
Speed: 0.0ms preprocess, 62.8ms i

In [ ]:
#캘리브레이션 윈도우사이즈 수정안함
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
import supervision as sv

class ObjectDetection:

    def __init__(self, capture_index, keyframe_interval, camera_matrix, dist_coeffs):

        self.capture_index = capture_index
        self.keyframe_interval = keyframe_interval
        self.K = camera_matrix
        self.D = dist_coeffs
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.1)
    
        self.keyframe_detected = False
    

    def load_model(self):
       
        model = YOLO("best.pt")  # enter path for a custom trained YOLOv8n model
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        # Setup detections for visualization
        detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.7f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        
        return frame
    

    def undistort(self, frame):
        h, w = frame.shape[:2]
        newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(self.K, self.D, (w, h), 1, (w, h))
        frame = cv2.undistort(frame, self.K, self.D, None, newcameramatrix)
        x, y, w, h = roi
        frame = frame[y:y+h, x:x+w]
        return frame


    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        image_index = 0  # Counter for saving frames

        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            frame = self.undistort(frame)  # 어안렌즈 보정
            
            if self.keyframe_detected or cap.get(cv2.CAP_PROP_POS_FRAMES) % self.keyframe_interval == 0:
                self.keyframe_detected = True
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)
            else:
                self.keyframe_detected = False

            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)

            #FPS 표시 
            #cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

            cv2.imshow('YOLOv8 Detection', frame)
            
            # Save each frame as an image
            cv2.imwrite(f'frame_{image_index}.png', frame)
            image_index += 1

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()
        cv2.destroyAllWindows()

focal_length = 1000  # 예시로 임의로 설정한 값
principal_point_x = 1020  # 예시로 임의로 설정한 값
principal_point_y = 760  # 예시로 임의로 설정한 값
k1, k2, p1, p2, k3, k4, k5, k6 = 0, 0, 0, 0, 0, 0, 0, 0  # 예시로 임의로 설정한 값

camera_matrix = np.array([[434.65221747,0.,304.1506615 ],
 [  0.,445.46687396, 291.03864346],
 [  0., 0.,1. ]]
)
dist_coeffs = np.array([[-0.72503986 , 0.62356206 , 0.00102756 , 0.01607157, -0.28152773]])

detector = ObjectDetection(capture_index=0, keyframe_interval=5, camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)
detector()


In [2]:
#칼리브레이션 없음
import torch
import numpy as np
import cv2
import os
from time import time
from ultralytics import YOLO
import supervision as sv

class ObjectDetection:

    def __init__(self, capture_index, keyframe_interval):
       
        self.capture_index = capture_index
        self.keyframe_interval = keyframe_interval
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.3)
    
        self.keyframe_detected = False
    

    def load_model(self):
       
        model = YOLO("best.pt")  # enter path for a custom trained YOLOv8n model
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        # Setup detections for visualization
        detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.7f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        
        return frame
    

    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        # Specify the frame rate at which to save images
        save_frame_rate = 10  # Save every 10th frame

        # Specify the directory to save the images
        save_directory = '0615resul'

        # Check if the directory exists, if not, create it
        if not os.path.exists(save_directory):
            os.makedirs(save_directory)

        frame_number = 0
      
        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            if self.keyframe_detected or frame_number % self.keyframe_interval == 0:
                self.keyframe_detected = True
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)
                
            else:
                self.keyframe_detected = False
            
            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)
             
            cv2.putText(frame, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            cv2.imshow('YOLOv8 Detection', frame)
            
            # Save frames as images at the specified frame rate
            if frame_number % save_frame_rate == 0:
                image_name = f'{save_directory}/frame_{frame_number}.jpg'
                cv2.imwrite(image_name, frame)
                print(f'Saved frame {frame_number} as {image_name}')
            
            frame_number += 1
 
            if cv2.waitKey(1) & 0xFF == ord('q'):
                 break
        
        cap.release()
        cv2.destroyAllWindows()

detector = ObjectDetection(capture_index=0 , keyframe_interval=5)# detect on keyframes every 30 frames
detector()



Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


Using Device:  cpu



0: 480x640 (no detections), 119.1ms
Speed: 2.0ms preprocess, 119.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 100.4ms
Speed: 2.2ms preprocess, 100.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



Saved frame 0 as 0615resul/frame_0.jpg


0: 480x640 (no detections), 114.7ms
Speed: 2.0ms preprocess, 114.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 97.6ms
Speed: 2.5ms preprocess, 97.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 116.2ms
Speed: 0.0ms preprocess, 116.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 94.9ms
Speed: 1.0ms preprocess, 94.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 114.6ms
Speed: 0.7ms preprocess, 114.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 101.4ms
Speed: 0.0ms preprocess, 101.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 106.5ms
Speed: 2.0ms preprocess, 106.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 101.0ms
Speed: 1.0ms preprocess

...

0: 480x640 1 photoshoot, 101.1ms
Speed: 0.0ms preprocess, 101.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 photoshoot, 93.7ms
Speed: 0.0ms preprocess, 93.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 photoshoot, 97.1ms
Speed: 0.0ms preprocess, 97.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 photoshoot, 98.2ms
Speed: 1.2ms preprocess, 98.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 2 photoshoots, 86.6ms
Speed: 0.0ms preprocess, 86.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 photoshoot, 97.9ms
Speed: 0.0ms preprocess, 97.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 1 photoshoot, 102.9ms
Speed: 0.0ms preprocess, 102.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 100.2ms
Speed: 0.0ms preprocess, 100.2ms inference, 0.0ms

Saved frame 350 as 0615resul/frame_350.jpg


0: 480x640 (no detections), 108.5ms
Speed: 0.0ms preprocess, 108.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 86.2ms
Speed: 0.0ms preprocess, 86.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 114.7ms
Speed: 1.0ms preprocess, 114.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 98.1ms
Speed: 1.0ms preprocess, 98.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 103.7ms
Speed: 0.0ms preprocess, 103.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 114.3ms
Speed: 1.0ms preprocess, 114.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 104.3ms
Speed: 1.1ms preprocess, 104.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 108.7ms
Speed: 0.0ms preprocess

Saved frame 360 as 0615resul/frame_360.jpg


0: 480x640 (no detections), 122.7ms
Speed: 0.0ms preprocess, 122.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 112.6ms
Speed: 0.0ms preprocess, 112.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 99.2ms
Speed: 0.0ms preprocess, 99.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 89.5ms
Speed: 0.0ms preprocess, 89.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 112.2ms
Speed: 1.0ms preprocess, 112.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 109.0ms
Speed: 0.0ms preprocess, 109.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 86.6ms
Speed: 0.0ms preprocess, 86.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 480x640 (no detections), 96.7ms
Speed: 1.0ms preprocess, 9

In [ ]:
import torch
import numpy as np
import cv2
from time import time
from ultralytics import YOLO
import supervision as sv

class ObjectDetection:

    def __init__(self, capture_index, keyframe_interval, camera_matrix, dist_coeffs):

        self.capture_index = capture_index
        self.keyframe_interval = keyframe_interval
        self.K = camera_matrix
        self.D = dist_coeffs
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print("Using Device: ", self.device)
        
        self.model = self.load_model()
        
        self.CLASS_NAMES_DICT = self.model.model.names
    
        self.box_annotator = sv.BoxAnnotator(sv.ColorPalette.default(), thickness=1, text_thickness=1, text_scale=0.5)
    
        self.keyframe_detected = False
        self.image_index = 0
    

    def load_model(self):
       
        model = YOLO("best.pt")  # enter path for a custom trained YOLOv5 model
        model.fuse()
    
        return model


    def predict(self, frame):
       
        results = self.model(frame)
        
        return results
    

    def plot_bboxes(self, results, frame):
        
        # Setup detections for visualization
        detections = sv.Detections(
                    xyxy=results[0].boxes.xyxy.cpu().numpy(),
                    confidence=results[0].boxes.conf.cpu().numpy(),
                    class_id=results[0].boxes.cls.cpu().numpy().astype(int),
                    )
        
    
        # Format custom labels
        self.labels = [f"{self.CLASS_NAMES_DICT[class_id]} {confidence:0.7f}"
        for _, confidence, class_id, tracker_id
        in detections]
        
        # Annotate and display frame
        frame = self.box_annotator.annotate(scene=frame, detections=detections, labels=self.labels)
        
        return frame
    

    def undistort(self, frame):
        h, w = frame.shape[:2]
        newcameramatrix, roi = cv2.getOptimalNewCameraMatrix(self.K, self.D, (w, h), 1, (w, h))
        frame = cv2.undistort(frame, self.K, self.D, None, newcameramatrix)
        x, y, w, h = roi
        frame = frame[y:y+h, x:x+w]
        return frame


    def __call__(self):

        cap = cv2.VideoCapture(self.capture_index)
        assert cap.isOpened()
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

        while True:
          
            start_time = time()
            
            ret, frame = cap.read()
            assert ret
            
            frame = self.undistort(frame)  # 어안렌즈 보정
            
            if cap.get(cv2.CAP_PROP_POS_FRAMES) % self.keyframe_interval == 0:
                results = self.predict(frame)
                frame = self.plot_bboxes(results, frame)

                # 클래스가 "photoshoot"인 경우 이미지 디스플레이 활성
                if "photoshoot" in self.labels:
                    cv2.imshow('YOLOv8 Detection', frame)
                    cv2.waitKey(0)
                    cv2.destroyAllWindows()
                    break

            end_time = time()
            fps = 1 / np.round(end_time - start_time, 2)

            cv2.imshow('YOLOv8 Detection', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        cap.release()

        # "mosaic_result.jpg" 디스플레이
        mosaic_img = cv2.imread('mosaic_result.jpg')
        cv2.imshow('Mosaic Result', mosaic_img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()


focal_length = 1000  # 예시로 임의로 설정한 값
principal_point_x = 1020  # 예시로 임의로 설정한 값
principal_point_y = 760  # 예시로 임의로 설정한 값
k1, k2, p1, p2, k3, k4, k5, k6 = 0, 0, 0, 0, 0, 0, 0, 0  # 예시로 임의로 설정한 값

camera_matrix = np.array([[434.65221747, 0., 304.1506615],
                          [0., 445.46687396, 291.03864346],
                          [0., 0., 1.]]
                         )
dist_coeffs = np.array([[-0.72503986, 0.62356206, 0.00102756, 0.01607157, -0.28152773]])

detector = ObjectDetection(capture_index=0, keyframe_interval=5, camera_matrix=camera_matrix, dist_coeffs=dist_coeffs)
detector()
